In [1]:
import z3

# Example Generator Matrix G (2 x 4)
G = [
    [1, 0, 1, 1],
    [0, 1, 1, 0]
]

k = len(G)      # number of logical qubits
n = len(G[0])   # number of physical qubits

def hamming_weight(codeword):
    return sum([z3.If(bit, 1, 0) for bit in codeword])

def find_minimum_distance(G):
    k = len(G)
    n = len(G[0])

    dmin = 1
    dmax = n

    while dmin < dmax:
        d = (dmin + dmax) // 2

        s = z3.Solver()
        ctx = s.ctx

        # Boolean vars for basis selection
        x = [z3.Bool(f"x_{i}") for i in range(k)]

        print(f"Checking for minimum distance d = {d}...")
        print("Variables:", x)

        # Generate codeword = x0 * G[0] ⊕ x1 * G[1] ⊕ ...
        codeword = [z3.BoolVal(False) for _ in range(n)]

        for i in range(k):
            for j in range(n):
                if G[i][j] == 1:
                    codeword[j] = z3.Xor(codeword[j], x[i])

        # Add constraint: at least one x_i is True (non-zero vector)
        s.add(z3.Or(x))

        # Add constraint: hamming weight ≤ d
        s.add(hamming_weight(codeword) <= d)

        if s.check() == z3.sat:
            dmax = d
            model = s.model()
            print(f"SAT for d = {d}: ", [model.evaluate(bit) for bit in codeword])
        else:
            dmin = d + 1

    return dmin

d = find_minimum_distance(G)
print("Minimum distance d =", d)



Checking for minimum distance d = 2...
Variables: [x_0, x_1]
SAT for d = 2:  [False, True, True, False]
Checking for minimum distance d = 1...
Variables: [x_0, x_1]
Minimum distance d = 2
